# Imports

In [149]:
import Pkg
basedir = "../"
Pkg.activate(basedir)
using BenchmarkTools

input = readlines(basedir*"data/input_05.txt")
input_test = readlines(basedir*"data/input_05t.txt");

  Activating project at `/uufs/chpc.utah.edu/common/home/sdss42/sdsswork/users/u6039752-1/software/AdventOfCode2023.jl`


# Part 1

In [5]:
function get_maps(input)
    map_splits = findall(input.=="")
    map_lsts = []
    for i=1:length(map_splits)
        if i < length(map_splits)
            push!(map_lsts,input[(map_splits[i]+2):(map_splits[i+1]-1)])
        else
            push!(map_lsts,input[(map_splits[i]+2):end])
        end
    end
    return map_lsts
end

function translate_map(old_prop_in,map_current)
    new_prop_out = copy(old_prop_in)
    for i=1:length(map_current)
        dest_start, source_start, rlen = parse.(Int,split(map_current[i]," "))
        msk = (source_start .<= old_prop_in .<= (source_start+rlen-1))
        if count(msk) != 0
            ind2change = findall(msk)
            for ind in ind2change
                refindx = findfirst((source_start:(source_start+rlen-1)).==old_prop_in[ind])
                new_prop_out[ind] = (dest_start:(dest_start+rlen-1))[refindx]
            end
        end 
    end
    return new_prop_out
end

function chain_translate(seeds,map_lsts)
    property_vec = []
    push!(property_vec,seeds)
    for (indx, map_current) in enumerate(map_lsts)
        push!(property_vec,translate_map(property_vec[indx],map_current))
    end
    return property_vec
end

seeds = parse.(Int,split(input_test[1]," ")[2:end])
map_lsts = get_maps(input_test)
property_vec = chain_translate(seeds,map_lsts)
minimum(property_vec[end])

seeds = parse.(Int,split(input[1]," ")[2:end])
map_lsts = get_maps(input)
property_vec = chain_translate(seeds,map_lsts)
minimum(property_vec[end])

function solution(input)
    seeds = parse.(Int,split(input[1]," ")[2:end])
    map_lsts = get_maps(input)
    property_vec = chain_translate(seeds,map_lsts)
    return minimum(property_vec[end])
end

solution(input)

457535844

# Part 2

In [7]:
function make_seed_tuples(input)
    seeds = parse.(Int,split(input[1]," ")[2:end])
    seed_tuples = []
    for i=1:(length(seeds)÷2)
        push!(seed_tuples,(seeds[2*(i-1)+1],seeds[2*(i-1)+2]))
    end
    return seed_tuples
end

make_seed_tuples (generic function with 1 method)

In [ ]:
        # low_end = maximum([sourcetup[1],intup[1]])
        # low_offset = low_end - sourcetup[1]
        # high_end = minimum([sourcetup[1]+sourcetup[2]-1,intup[1]+intup[2]-1])
        # high_offset = high_end - (sourcetup[1]+sourcetup[2]-1)

In [153]:
input_tup = make_seed_tuples(input_test)
map_lsts = get_maps(input_test)
dest_start, source_start, rlen = parse.(Int,split(map_lsts[1][1]," "))
sourcetup = (source_start,rlen)
desttup = (dest_start,rlen)
trans_lst, same_lst = transform_tuple(input_tup[2],sourcetup,desttup)

(Any[], [(55, 13)])

In [101]:
input_tup[1], sourcetup

((79, 14), (98, 2))

In [102]:
input_tup[1][1]+input_tup[1][2]-1

92

In [103]:
input_tup[1]

(79, 14)

In [104]:
sourcetup[1]+sourcetup[2]-1

99

In [105]:
out_tup, offset_tup = transform_tuple((10,2),(5,10),(20,10))

([(25, 2)], Any[])

In [106]:
out_tup, offset_tup = transform_tuple((2,10),(5,10),(20,10))

([(20, 7)], [(2, 3)])

In [107]:
transform_tuple((10,10),(5,10),(20,10))

([(25, 5)], [(15, 5)])

In [111]:
transform_tuple((2,20),(5,10),(20,10))

([(20, 10)], [(2, 3), (15, 7)])

In [143]:
function transform_tuple(intup,sourcetup,desttup)
    out_lst = []
    if (sourcetup[1] <= intup[1]) & (intup[1]+intup[2]-1 <= sourcetup[1]+sourcetup[2]-1)
        dvec = intup[1] - sourcetup[1]
        return [(desttup[1]+dvec, intup[2])], []
    elseif (intup[1] <= sourcetup[1]) & (sourcetup[1] <= intup[1]+intup[2]-1 <= sourcetup[1]+sourcetup[2]-1)
        dvec = sourcetup[1]-intup[1]
        return [(desttup[1],intup[2]-dvec)],[(intup[1],dvec)]
    elseif (sourcetup[1] <= intup[1] <= sourcetup[1]+sourcetup[2]-1) & (sourcetup[1]+sourcetup[2]-1 <= intup[1]+intup[2]-1)
        dvec = sourcetup[1]+sourcetup[2]-intup[1]
        return [(desttup[1]+desttup[2]-dvec,dvec)], [(intup[1]+dvec,intup[2]-dvec)]
    elseif (intup[1] <= sourcetup[1]) & (sourcetup[1]+sourcetup[2]-1 <= intup[1]+intup[2]-1)
        dvec_left = sourcetup[1]-intup[1]
        dvec_right = sourcetup[1]+sourcetup[2]-intup[1]
        return [desttup], [(intup[1],dvec_left),(intup[1]+dvec_right,intup[2]-dvec_right)]
    else
        return [], [intup]
    end
end

transform_tuple (generic function with 1 method)

In [141]:
function translate_tup_via_map(old_prop_in_tup,map_current)
    old_tup_lst = old_prop_in_tup
    new_tup_lst = []
    for mapindx=1:length(map_current)
        dest_start, source_start, rlen = parse.(Int,split(map_current[mapindx]," "))
        sourcetup = (source_start,rlen)
        desttup = (dest_start,rlen)
        trans_lst_outer = []
        same_lst_outer = []
        for i=1:length(old_tup_lst)
            trans_lst, same_lst = transform_tuple(old_tup_lst[i],sourcetup,desttup)
            push!(trans_lst_outer,trans_lst)
            push!(same_lst_outer,same_lst)
        end
        old_tup_lst = vcat(same_lst_outer...)
        push!(new_tup_lst,trans_lst_outer...)
    end
    push!(new_tup_lst,old_tup_lst...)
    result_lst = vcat(new_tup_lst...)
    return result_lst
end

translate_tup_via_map (generic function with 1 method)

In [133]:
new_tup_lst

2-element Vector{Any}:
 Any[Any[], Any[]]
 Any[[(81, 14)], [(57, 13)]]

In [128]:
old_tup_lst

Any[]

In [129]:
new_tup_lst

2-element Vector{Any}:
 Any[Any[], Any[]]
 Any[[(81, 14)], [(57, 13)]]

In [125]:
old_tup_lst

Any[]

In [142]:
translate_tup_via_map(input_tup,map_lsts[1])

2-element Vector{Any}:
 (81, 14)
 (57, 13)

In [144]:
function chain_translate(seeds,map_lsts)
    property_vec = []
    push!(property_vec,seeds)
    for (indx, map_current) in enumerate(map_lsts)
        push!(property_vec,translate_tup_via_map(property_vec[indx],map_current))
    end
    return property_vec
end

chain_translate (generic function with 1 method)

In [154]:
tout = chain_translate(input_tup,map_lsts)

8-element Vector{Any}:
 Any[(79, 14), (55, 13)]
 Any[(81, 14), (57, 13)]
 Any[(81, 14), (57, 13)]
 Any[(53, 4), (81, 14), (61, 9)]
 Any[(46, 4), (74, 14), (54, 9)]
 Any[(45, 11), (82, 4), (90, 9), (78, 3)]
 Any[(46, 11), (82, 4), (90, 9), (78, 3)]
 Any[(60, 1), (86, 4), (94, 3), (82, 3), (56, 4), (46, 10), (97, 2)]

In [157]:
minimum(map(x->x[1],tout[end]))

46

In [158]:
input_tup = make_seed_tuples(input)
map_lsts = get_maps(input)
property_vec = chain_translate(input_tup,map_lsts)
minimum(map(x->x[1],property_vec[end]))

41222968

In [159]:
function solution(input)
    input_tup = make_seed_tuples(input)
    map_lsts = get_maps(input)
    property_vec = chain_translate(input_tup,map_lsts)
    return minimum(map(x->x[1],property_vec[end]))
end

solution(input)

41222968